In [1]:
using DataFrames, CSV, Query #Data handling
using Convex, GLPKMathProgInterface # Optimization tools

fn = "MOWOG1entries.csv"
c1_list=["AS", "BS", "CS", "DS", "ES", "FS", "GS", "HS","SS","SSR"] # Combined 1 classes
_run_groups = 2 #Number of run groups for the event
_max_to_bump = 4 #Maximum number of entrants in the class that will still be bumped to combined
_max_driver_diff = 4 #Maximum difference in number of drivers per run group
_max_novice_diff = 5 #Maximum difference in number of novice drivers per run group

#Distribute an integer over N integer parts
function distribute_int(a::T,n::T) where {T<:Integer}
    (num,den) = divrem(a,n)
    [ifelse(i<=den,num+1,num) for i=1:n]
end
@assert distribute_int(10,3)==[4,3,3]

function index_class(a::T) where {T<:DataFrameRow}
    if ismissing(a[:Index])
        if a[:Class]=="N"
            'N'*a[:LastName][1] |> String
        else
            a[:Class] |> String
        end
    else
        a[:Index] |> String
    end
end

function rungroup(a::Convex.AbstractExprOrValue,df::DataFrame, n::Integer)
    @assert n<=size(a,2)
    A = evaluate(a[:,n]) .≈ 1.0
    novice_ind = A .& df[:Novice]
    n1 = findfirst(novice_ind) |> i->df[i,:Class][2]
    n2 = findlast(novice_ind) |> i->df[i,:Class][2]
    ind = A .& .!(df[:Novice]) |> find
    y=df[ind,:Class]
    push!(y,"Novice $n1-$n2")
end

rungroup (generic function with 1 method)

In [2]:
df=CSV.read(fn); #Read the CSV to a DataFrame
rename!(df, Symbol("Modifier/PAX") => :Index)
# rename!(df, Symbol("First Name")=> :FirstName)
rename!(df, Symbol("Last Name")=> :LastName)
delete!(df, [Symbol("Segment Name"),:Group])
df[:IndexClass]=map(x->index_class(x),eachrow(df))
head(df)

,LastName,Class,Year,Make,Model,Index,IndexClass
1,Ag,GS,2013,Ford,Focus ST,Z,Z
2,An,HS,2015,Ford,Fiesta ST,Z,Z
3,Au,ES,2004,Toyota,MR2,P,P
4,Ba,BS,1992,Chevrolet,Corvette,missing,BS
5,Ba,STS,1988,Honda,CRX,P,P
6,Ba,N,1996,Lexus,sc400,missing,NB


Randomly assign 23 entrants an exempt work position for testing purposes
with a preset random seed

In [3]:
n_drivers=nrow(df);
exempt_drivers=fill(false,n_drivers);
srand(562161);
exempt_drivers[randperm(n_drivers)[1:23]]=true;
df[:Exempt]=exempt_drivers;
head(df)

,LastName,Class,Year,Make,Model,Index,IndexClass,Exempt
1,Ag,GS,2013,Ford,Focus ST,Z,Z,false
2,An,HS,2015,Ford,Fiesta ST,Z,Z,false
3,Au,ES,2004,Toyota,MR2,P,P,true
4,Ba,BS,1992,Chevrolet,Corvette,missing,BS,false
5,Ba,STS,1988,Honda,CRX,P,P,false
6,Ba,N,1996,Lexus,sc400,missing,NB,false


In [4]:
novice_fill_df=DataFrame(Class='N'.*('A':'Z'), Drivers=0,Exempt=0);

In [5]:
#Count up the drivers per class
df=@from i in df begin
    @group i by i.IndexClass into g
    @orderby ascending(g.key)
    @select {Class=g.key, Drivers=length(g), Exempt=sum(g..Exempt)}
    @collect DataFrame
end

#Fill in the empty novice classes, and reduce
append!(df,novice_fill_df)
df=@from i in df begin
    @group i by i.Class into g
    @orderby ascending(g.key)
    @select {Class=g.key, Drivers=sum(g..Drivers), Exempt=sum(g..Exempt), Novice=g.key[1]=='N'}
    @collect DataFrame
end
head(df)

,Class,Drivers,Exempt,Novice
1,ASP,1,0,false
2,BS,7,1,false
3,CAM-S,3,2,false
4,CAM-T,3,1,false
5,CS,3,1,false
6,DM,2,0,false


In [6]:
# let d=df[find(x->x[:Class]=="N",eachrow(df)),:]
#     DataFrame(Class=(@. "N"*string(1:_run_groups)),
#         Drivers=distribute_int(d[:Drivers][1],_run_groups),
#         Exempt=distribute_int(d[:Exempt][1],_run_groups)) |> x -> append!(df,x)
# end

df=@from i in df begin
    @where i.Class != "N"
    @select i
    @collect DataFrame
end
df[:Workers]=df[:Drivers].-df[:Exempt]
head(df)

,Class,Drivers,Exempt,Novice,Workers
1,ASP,1,0,false,1
2,BS,7,1,false,6
3,CAM-S,3,2,false,1
4,CAM-T,3,1,false,2
5,CS,3,1,false,2
6,DM,2,0,false,2


In [7]:
# Create our variables
N = nrow(df)
x = Variable((N,_run_groups), :Bin) #Class allocation variable

Variable of
size: (50, 2)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()

In [8]:
#Each class must be in exactly 1 run group
constr=sum(x,2).==1;

This next set constrains the Novice class split so that Run Group #1 starts a A and continues to a "L1",
and Group #2 resumes form "L1+1" to "L2", and Group #3 etc... resumes from "L2+1" to Z.

An illustration of the constraints is shown below, for an example of 4 letters, and 3 run groups

In [9]:
let N=4, _run_groups=3
    ["sum(x[$i,1:$n])>=x[$(i+1),$n]" for n=1:_run_groups-1 for i=1:N-1] .|> println
end;

sum(x[1,1:1])>=x[2,1]
sum(x[2,1:1])>=x[3,1]
sum(x[3,1:1])>=x[4,1]
sum(x[1,1:2])>=x[2,2]
sum(x[2,1:2])>=x[3,2]
sum(x[3,1:2])>=x[4,2]


In [10]:
sub_x = x[df[:Novice] |> find,:]  # sub_array of only the novice classes
constr+=(x->[sum(x[i,1:n])>=x[(i+1),n] for n=1:_run_groups-1 for i=1:size(x,1)-1])(sub_x);

In [11]:
#keep Combined classes together if necessary
constr+=let ind=[any(d[:Class].==c1_list) && (d[:Drivers] <= _max_to_bump) && !d[:Novice] for d in eachrow(df)] |> find
    "Combining $(join(df[ind,:Class],',')) due to <= $_max_to_bump drivers" |> println
    [x[ind[1:end-1],run_group].==x[ind[2:end],run_group] for run_group=1:_run_groups]
end;
constr+=let ind=[!any(d[:Class].==c1_list) && (d[:Drivers] <= _max_to_bump) && !d[:Novice] for d in eachrow(df)] |> find
    "Combining $(join(df[ind,:Class],',')) due to <= $_max_to_bump drivers" |> println
    [x[ind[1:end-1],run_group].==x[ind[2:end],run_group] for run_group=1:_run_groups]
end;

Combining CS,DS,FS,GS,HS due to <= 4 drivers
Combining ASP,CAM-S,CAM-T,DM,EM,SSC,SSM,SSP,STH,STS,STU,STX,V,X due to <= 4 drivers


In [12]:
#Split Pro & Z
constr+=let ind=[any(d[:Class].==["P","Z"]) for d in eachrow(df)] |> find
    sum(x[ind,:],1).<=1
end;

In [13]:
#Expressions that can be used in the optimizer
rungroup_workers=sum(x.*df[:Workers],1) |> vec #Workers available per run group
rungroup_drivers=sum(x.*df[:Drivers],1) |> vec #Drivers in each run group
rungroup_novice= sum(Vector(df[:Drivers].*df[:Novice]).*x,1) |> vec

AbstractExpr with
head: reshape
size: (2, 1)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()


In [14]:
constr+=maximum(rungroup_drivers)-minimum(rungroup_drivers)<=_max_driver_diff;
constr+=maximum(rungroup_novice)-minimum(rungroup_novice)<=_max_novice_diff;

In [15]:
# Define the problem's optimization, under required constraints
p=maximize(minimum(rungroup_workers),constr);  #Maximize, the Minimum # of workers in a run group

In [16]:
solve!(p, GLPKSolverMIP())

In [17]:
#What is the status of the solutioin
p.status

:Optimal

In [18]:
let f = x-> Integer.(evaluate(x))
    for (i,(drivers,novice,workers)) in enumerate(zip(f.((rungroup_drivers,rungroup_novice,rungroup_workers))...))
        println("Run group #$i, $drivers drivers, $workers workers, $novice novice drivers")
    end
end

Run group #1, 68 drivers, 58 workers, 17 novice drivers
Run group #2, 71 drivers, 58 workers, 12 novice drivers


In [19]:
println("Run Group #1:"); println.(rungroup(x,df,1));

Run Group #1:
BS
CS
DS
ES
FS
GS
HS
P
SMF
Novice A-L


In [20]:
println("Run Group #2:"); println.(rungroup(x,df,2));

Run Group #2:
ASP
CAM-S
CAM-T
DM
EM
SSC
SSM
SSP
STH
STS
STU
STX
V
X
Z
Novice M-Z


In [21]:
_run_groups >= 3 && (println("Run Group #3:"); println.(rungroup(x,df,3)));